### **Proyecto 3 - Ingesta, Transformación, Carga y Reportería**

#### **Arquitectura del Proyecto**

<center><img src="https://i.postimg.cc/t4j2B2gs/adf455.png"></center>

### **Paso 1 - Data Ingestion**
_____

1. Pasos a seguir en nuestra Ingesta de datos desde Azure Blob Storage hacia Azure Data Lake Storage Gen2

    *   Crear un contenedor en ABS donde almancenar la data
    *   Crear un contenedor en ADLS donde almacenar la data
    *   Crear Linked Services de origen y destino
    *   Crear datasets de origen y destino    
    *   Crear un Pipeline de ingesta de datos    


<center><img src="https://images2.imgbox.com/a7/8b/zClZtz13_o.png"></center> <!-- adf664 -->

2. Pasos a seguir en nuestra Ingesta de datos desde HTTP hacia Azure Data Lake Storage Gen2

    *   Reutilizaremos contenedor en ADLS donde almacenar la data
    *   Crear un Linked Service de origen y reutilizar el Linked Service de destino
    *   Crear datasets parametrizados tanto de origen como destino 
    *   Crear un Pipeline de ingesta de datos parametrizado    


<center><img src="https://images2.imgbox.com/3e/19/JBr4g1NM_o.png"></center> <!-- adf721 -->

##### **Paso 1.1 - Creación de Linked Services de origen y destino para la ingesta de datos desde Azure Blob Storage hacia Azure Data Lake Storage Gen2**

1. Crear el Linked Service de origen **ls_ablob_covidreportingsa** que hace referencia a Azure Blob Storage

2. Crear el Linked Service de destino **ls_adls_covidreportingdl** que hace referencia a nuestro ADLS

##### **Paso 1.2 - Creación de Datasets de origen y destino para la ingesta de datos desde Azure Blob Storage hacia Azure Data Lake Storage Gen2**

1. Crear el Dataset de origen **ds_population_raw_gz** que hace referencia al archivo **population_by_age.tsv.gz** alojado en el contenedor **population** en Azure Blob Storage

2. Crear el Dataset de destino **ds_population_raw_tsv** que hace referencia al archivo **population_by_age.tsv** (que aún no existe, pero se   
   creará de manera automática al ejecutar el pipeline) a nuestro ADLS, a la ruta **raw/population**. Siendo **raw** el contenedor y **population** el directorio

##### **Paso 1.3 - Creación de un Pipeline para la ingesta de datos desde Azure Blob Storage hacia Azure Data Lake Storage Gen2**

1. Crear el Pipeline de ingesta de datos **pl_ingest_population_data**, el cual contendrá una actividad **Copy data**

##### **Paso 1.4 - Escenarios extras**

1. Ejecutar la actividad **Copy data** cuando el archivo esté disponible

2. Ejecutar la actividad **Copy data** sólo si el contenido del archivo es el esperado

3. Eliminar el archivo de origen al copiarlo correctamente

##### **Paso 1.5 - Creación de un Linked Service de origen para la ingesta de datos desde HTTP hacia Azure Data Lake Storage Gen2**

1. Crear el Linked Service de origen **ls_http_opendata_ecdc_europa_eu** que hace referencia a HTTP

##### **Paso 1.6 - Creación de Datasets de origen y destino parametrizados para la ingesta de datos desde HTTP hacia Azure Data Lake Storage Gen2**

1. Crear el Dataset de origen **ds_ecdc_raw_csv_http** que hace referencia al archivo parametrizado **@dataset().relativeURL**. Dicho valor del   
   parámetro será indicado al momento de ejecutar el pipeline

2. Crear el Dataset de destino **ds_ecdc_raw_csv_dl** que hace referencia al archivo parametrizado **@dataset().fileName** (que aún no existe,    
   pero se creará de manera automática al ejecutar el pipeline) de nuestro ADLS, aque se almacenará en la ruta **raw/ecdc**. Siendo **raw** el contenedor y **ecdc** el directorio

##### **Paso 1.7 - Creación de un Pipeline parametrizado para la ingesta de datos desde HTTP hacia Azure Data Lake Storage Gen2**

1. Crear el Pipeline de ingesta de datos parametrizado **pl_ingest_ecdc_data**, el cual contendrá una actividad **Copy data**

##### **Paso 1.8 - Creación de un Trigger "Schedule" que se anexará al pipeline "pl_ingest_ecdc_data"**

1. Crear el Trigger **Schedule** y que se llamará **tr_ingest_hospital_admissions_data** y se anexará al pipeline **pl_ingest_ecdc_data**. Al   
   ejecutarse el trigger solo ingestará datos de **hospital_admissions.csv**

##### **Paso 1.9 - Parametrizar el Linked Service de origen "ls_http_opendata_ecdc_europa_eu" para la ingesta de datos desde HTTP hacia Azure Data Lake Storage Gen2**

1. Parametrizar el Linked Service de origen **ls_http_opendata_ecdc_europa_eu**

2. Crear un nuevo parámetro en el dataset de origen **ds_ecdc_raw_csv_http**

3. Crear un nuevo parámetro en el pipeline **pl_ingest_ecdc_data**

##### **Paso 1.10 - Modificación final al proceso de ingesta desde HTTP**

### **Paso 2 - Data Transformation (Data Flow)**
_____

1. Pasos en la transformación de datos del archivo **cases_deaths.csv**

    *   Crear un dataflow para el proceso de transformación

<center><img src="https://i.postimg.cc/MKvdgTnT/adf780.png"></center>

2. Pasos en la transformación de datos del archivo **hospital-admissions.csv**

    *   Crear un dataflow para el proceso de transformación

<center><img src="https://i.postimg.cc/5tqD1hMN/adf848.png"></center>
<center><img src="https://i.postimg.cc/C1hXKcFC/adf849.png"></center>

##### **Paso 2.1 - Creamos el Data flow "df_transform_cases_deaths" para transformar los datos del archivo "cases_deaths.csv"**

1. Configurar el stream **Source** y crear un nuevo dataset de origen que haga referencia al archivo **cases_deaths.csv**, llamado  
   **ds_raw_cases_and_deaths**

2. Agregar un stream **Filter** para filtrar el campo **continent** y nos devuelva solo registros para el continente **Europe**

3. Agregar un stream **Select** para seleccionar las columnas que necesitamos

4. Agregar un stream **Pivot**

5. Agregar un stream **Lookup** y un stream **Source**. Además, crear un nuevo dataset de origen que haga referencia al archivo **country_lookup.csv**, llamado **ds_country_lookup**

6. Nuevamente, agregar un stream **Select** para seleccionar y ordenar las columnas que necesitamos   

7. Y para finalizar, utilizaremos un stream **Sink** y creamos un un nuevo dataset de destino que haga referencia a la ruta **processed/ecdc/cases_deaths** de nuestro ADLS. No se indicó un nombre de archivo

8. Creamos un pipeline llamado **pl_process_cases_and_deaths_data*** para ejecutar el Data flow recién creado

##### **Paso 2.2 - Creamos el Data flow "df_transform_cases_deaths" para transformar los datos del archivo "hospital-admissions.csv"**

1. Configurar el stream **Source** y crear un nuevo dataset de origen que haga referencia al archivo **hospital-admissions.csv**, llamado  
   **ds_raw_hospital_admission**

2. Agregar un stream **Select** para renombrar y eliminar campos

3. Agregar un stream **Lookup** y un stream **Source**. El stream **Source** hace referencia al archivo **country_lookup.csv**, más especificamente al dataset **ds_country_lookup**

4. Agregar un stream **Select** para eliminar un campo

5. Agregar un stream **Split Conditional**

6. Agregar y configurar un nuevo stream **Source** y crear un nuevo dataset de origen que haga referencia al archivo **dim_date.csv**, llamado **ds_dim_date_lookup** 

7. Agregar un stream **Derived Column** para crear un nuevo campo

8. Agregar un stream **Aggregate**

9. Agregar un stream **Join**

10. Agregar un stream **Pivot** llamado **PivotWeekly**

11. Agregar un stream **Pivot** llamado **PivotDaily**

12. Agregar un stream **Sort** llamado **SortWeekly**

13. Agregar un stream **Sort** llamado **SortDaily**

14. Agregar un stream **Select** llamado **SelectWeekly** para renombrar columnas

15. Agregar un stream **Select** llamado **SelectDaily** para renombrar columnas

7. Y para finalizar, utilizaremos un stream **Sink** llamado **WeeklySink**  y creamos un un nuevo dataset de destino que haga referencia a la ruta **processed/ecdc/hospital_admission_weekly** de nuestro ADLS. No se indicó un nombre de archivo. Y un stream **Sink** llamado **DailySink**  y creamos un un nuevo dataset de destino que haga referencia a la ruta **processed/ecdc/hospital_admission_daily** de nuestro ADLS. Tampoco se indicó un nombre de archivo.

8. Creamos el pipeline **pl_process_hospital_admissions_data** para ejecutar el Data flow recién creado

##### **Paso 2.3 - Modificaciones**

1. Modificar el dataset **ds_ecdc_file_list**

2. Modificar el dataset **ds_raw_cases_and_deaths**

2. Modificar el dataset **ds_raw_hospital_admission**

2. Modificar el dataset **ds_country_lookup**

3. Modificar el dataset **ds_dim_date_lookup**

### **Paso 3 - Data Transformation (HDInsight)**
_____

##### **Paso 3.1 - Utilizar un Cluster HDInsight para transformar datos utilizando HIVE**

1. Crear un recurso **User Assigned Managed Identity** 

2. Generar acceso basado en un rol al recurso de Managed Identity que hemos creado en el ADLS **covidreportingdl**

3. Crear un recurso **Azure HDInsight Cluster**

4. Crear el pipeline **pl_process_testing_data** con una actividad **Hive**. También, crear el Linked Service **ls_hdi_covid_cluster** que haga referencia al Cluster HDInsight

### **Paso 4 - Data Transformation (Databricks)**
_____

##### **Paso 4.1 - Mount storage**

##### **Paso 4.2 - Transformar los datos del archivo population.tsv**

##### **Paso 4.3 - Crear el pipeline que ejecute un notebook de Databricks**

1. Crear el Linked Service **ls_db_covid_cluster** que haga referencia a Databricks. Utilizaremos una autenticación del tipo **Access Token**.

2. Crear el pipeline **pl_process_population_data** con una actividad **Notebook**